In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (StandardScaler, PolynomialFeatures)
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score, f1_score, roc_auc_score,recall_score, log_loss
from sklearn.metrics import confusion_matrix

In [2]:
data= pd.read_csv('train.csv')
data.describe()
#try to remove loan title as it contains many uncleaned data records

,ID,Loan Amount,Funded Amount,Funded Amount Investor,Term,Interest Rate,Home Ownership,Debit to Income,Delinquency - two years,Inquires - six months,...,Total Received Late Fee,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Last week Pay,Accounts Delinquent,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
count,6.746300e+04,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,...,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.0,67463.000000,6.746300e+04,67463.000000,67463.000000
mean,2.562761e+07,16848.902776,15770.599114,14621.799323,58.173814,11.846258,80541.502522,23.299241,0.327127,0.145754,...,1.143969,59.691578,1.125141,0.021301,71.163260,0.0,146.467990,1.595739e+05,23123.005544,0.092510
std,2.109155e+07,8367.865726,8150.992662,6785.345170,3.327441,3.718629,45029.120366,8.451824,0.800888,0.473291,...,5.244365,357.026346,3.489885,0.144385,43.315845,0.0,744.382233,1.390332e+05,20916.699999,0.289747
min,1.297933e+06,1014.000000,1014.000000,1114.590204,36.000000,5.320006,14573.537170,0.675299,0.000000,0.000000,...,0.000003,0.000036,0.000036,0.000000,0.000000,0.0,1.000000,6.170000e+02,1000.000000,0.000000
25%,6.570288e+06,10012.000000,9266.500000,9831.684984,58.000000,9.297147,51689.843335,16.756416,0.000000,0.000000,...,0.021114,1.629818,0.476259,0.000000,35.000000,0.0,24.000000,5.037900e+04,8155.500000,0.000000
50%,1.791565e+07,16073.000000,13042.000000,12793.682170,59.000000,11.377696,69335.832680,22.656658,0.000000,0.000000,...,0.043398,3.344524,0.780141,0.000000,68.000000,0.0,36.000000,1.183690e+05,16733.000000,0.000000
75%,4.271521e+07,22106.000000,21793.000000,17807.594120,59.000000,14.193533,94623.322785,30.048400,0.000000,0.000000,...,0.071884,5.453727,1.070566,0.000000,105.000000,0.0,46.000000,2.283750e+05,32146.500000,0.000000
max,7.224578e+07,35000.000000,34999.000000,34999.746430,59.000000,27.182348,406561.536400,39.629862,8.000000,5.000000,...,42.618882,4354.467419,166.833000,1.000000,161.000000,0.0,16421.000000,1.177412e+06,201169.000000,1.000000


In [3]:
data1= data.drop(['ID','Batch Enrolled','Accounts Delinquent','Payment Plan','Loan Title'], axis = 1)

In [4]:
#frmat Initial List Status, Application Type
data1['Initial List Status'] = np.where(data1['Initial List Status']=='w', 1, 0)
data1['Application Type'] = np.where(data1['Application Type']=='INDIVIDUAL', 1, 0)


In [5]:
print(data.nunique())
a =data1.head()

ID                              67463
Loan Amount                     27525
Funded Amount                   24548
Funded Amount Investor          67441
Term                                3
Batch Enrolled                     41
Interest Rate                   67448
Grade                               7
Sub Grade                          35
Employment Duration                 3
Home Ownership                  67454
Verification Status                 3
Payment Plan                        1
Loan Title                        109
Debit to Income                 67454
Delinquency - two years             9
Inquires - six months               6
Open Account                       36
Public Record                       5
Revolving Balance               20582
Revolving Utilities             67458
Total Accounts                     69
Initial List Status                 2
Total Received Interest         67451
Total Received Late Fee         67380
Recoveries                      67387
Collection R

In [6]:
#check nulls
column_list = data1.columns
for column in column_list:
    print('Number of null recs in col ',column,'is ',data1[column].isnull().sum())


Number of null recs in col  Loan Amount is  0
Number of null recs in col  Funded Amount is  0
Number of null recs in col  Funded Amount Investor is  0
Number of null recs in col  Term is  0
Number of null recs in col  Interest Rate is  0
Number of null recs in col  Grade is  0
Number of null recs in col  Sub Grade is  0
Number of null recs in col  Employment Duration is  0
Number of null recs in col  Home Ownership is  0
Number of null recs in col  Verification Status is  0
Number of null recs in col  Debit to Income is  0
Number of null recs in col  Delinquency - two years is  0
Number of null recs in col  Inquires - six months is  0
Number of null recs in col  Open Account is  0
Number of null recs in col  Public Record is  0
Number of null recs in col  Revolving Balance is  0
Number of null recs in col  Revolving Utilities is  0
Number of null recs in col  Total Accounts is  0
Number of null recs in col  Initial List Status is  0
Number of null recs in col  Total Received Interest i

In [7]:
data_ohc = data1.copy()

In [8]:
data_sample = data1.head()

In [9]:
#nunique- no of uniques
mask = data_ohc.dtypes == np.object
# we use this true false sequence to filter cols
categorical_cols = data_ohc.columns[mask]

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_15816/1442200654.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = data_ohc.dtypes == np.object


In [10]:
#easy method to one hot encode
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(sparse=False)

data_ohc_encoded = pd.DataFrame (ohe.fit_transform(data_ohc[categorical_cols]))

data_ohc_encoded.columns = ohe.get_feature_names(categorical_cols)

data_ohc.drop(categorical_cols ,axis=1, inplace=True)

data_ohc= pd.concat([data_ohc, data_ohc_encoded ], axis=1)

#data_ohc is final

ohc_sample = data_ohc.head()
data_ohc.shape[1] - data1.shape[1]

C:\Users\ADMIN\anaconda3\envs\2021_11_28_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


44

In [11]:
data_ohc_temp = data_ohc.head(1000)

In [29]:
y_col = 'Loan Status'

feature_cols = [x for x in data_ohc.columns if x != y_col]
X = data_ohc[feature_cols]
y = data_ohc[y_col]

In [23]:
# if we just want to test code
feature_cols = [x for x in data_ohc_temp.columns if x != y_col]
X = data_ohc_temp[feature_cols]
y = data_ohc_temp[y_col]

In [30]:
y[:].sum()

6241

In [17]:
#class weight
y[:].sum()
#9 1s per 10 zeros
w = {0:1, 1:9}

# Decision Tree

In [31]:
estimator = Pipeline([
                    ("scaler", StandardScaler()),
                    ("polynomial_features", PolynomialFeatures()),
                    ("DecisionTreeClassifier", DecisionTreeClassifier())
                    ])


params = {
                'polynomial_features__degree': [1,2],
                'DecisionTreeClassifier__criterion': ["gini", "entropy"],
                'DecisionTreeClassifier__class_weight': ["balanced"]
        }

#double underscore is used here
kf = KFold(shuffle=True, random_state=72018, n_splits=3)

#grid = GridSearchCV(estimator, params, cv=kf,scoring='f1' ,verbose = True)
grid = GridSearchCV(estimator, params,cv = kf ,scoring='f1' ,verbose = True)

grid.fit(X, y)
print(grid.best_score_ )
print(grid.best_params_)

print(grid.best_estimator_)
y_predict = grid.predict(X)

print('Confusion Matrix \n',confusion_matrix(y,y_predict))
print('Area Under Curve:', roc_auc_score(y,y_predict))
print('Recall score: ',recall_score(y,y_predict))
print('f1_score: ',f1_score(y,y_predict))
print('log_loss: ',log_loss(y,y_predict))

# {'DecisionTreeClassifier__class_weight': 'balanced', 'DecisionTreeClassifier__criterion': 'gini', 'polynomial_features__degree': 2}

Fitting 3 folds for each of 4 candidates, totalling 12 fits


KeyboardInterrupt: 

In [28]:
print(grid.best_params_)

{'DecisionTreeClassifier__class_weight': 'balanced', 'DecisionTreeClassifier__criterion': 'gini', 'polynomial_features__degree': 2}


# Ridge Classifier

In [ ]:
# Ridge classifier
estimator = Pipeline([
                    ("scaler", StandardScaler()),
                    ("polynomial_features", PolynomialFeatures()),
                    ("DecisionTreeClassifier", DecisionTreeClassifier())
                    ])


params = {
                'polynomial_features__degree': [1,2],
                'RidgeClassifier__alpha': [10]
}

#double underscore is used here
kf = KFold(shuffle=True, random_state=72018, n_splits=3)

#grid = GridSearchCV(estimator, params, cv=kf,scoring='f1' ,verbose = True)
grid = GridSearchCV(estimator, params,cv = kf ,scoring='f1' ,verbose = True)

grid.fit(X, y)
print(grid.best_score_ )
print(grid.best_params_)

print(grid.best_estimator_)
y_predict = grid.predict(X)

print('Confusion Matrix \n',confusion_matrix(y,y_predict))
print('Area Under Curve:', roc_auc_score(y,y_predict))
print('Recall score: ',recall_score(y,y_predict))
print('f1_score: ',f1_score(y,y_predict))
print('log_loss: ',log_loss(y,y_predict))

# {'DecisionTreeClassifier__class_weight': 'balanced', 
#'DecisionTreeClassifier__criterion': 'gini', 'polynomial_features__degree': 2}

# Logisic Regression

In [ ]:
# Logistic regression
estimator = Pipeline([
                    ("scaler", StandardScaler()),
                    ("polynomial_features", PolynomialFeatures()),
                    ("DecisionTreeClassifier", DecisionTreeClassifier())
                    ])


params = {
                'polynomial_features__degree': [1,2],
                'logistic_regression__max_iter':[100],
                'logistic_regression__C': np.logspace(-4, 4, 10),
                'logistic_regression__penalty' : ['l1'],
                'logistic_regression__solver' : ['liblinear']
}

#double underscore is used here
kf = KFold(shuffle=True, random_state=72018, n_splits=3)

#grid = GridSearchCV(estimator, params, cv=kf,scoring='f1' ,verbose = True)
grid = GridSearchCV(estimator, params,cv = kf ,scoring='f1' ,verbose = True)

grid.fit(X, y)
print(grid.best_score_ )
print(grid.best_params_)

print(grid.best_estimator_)
y_predict = grid.predict(X)

print('Confusion Matrix \n',confusion_matrix(y,y_predict))
print('Area Under Curve:', roc_auc_score(y,y_predict))
print('Recall score: ',recall_score(y,y_predict))
print('f1_score: ',f1_score(y,y_predict))
print('log_loss: ',log_loss(y,y_predict))

# {'DecisionTreeClassifier__class_weight': 'balanced', 
#'DecisionTreeClassifier__criterion': 'gini', 'polynomial_features__degree': 2}

# Check performance for each grid and store the best and use it for test dataset

# Loading test dataset

In [ ]:
#taking in test data set

test= pd.read_csv('test.csv')

test= test.drop(['ID','Batch Enrolled','Accounts Delinquent','Payment Plan','Loan Title'], axis = 1)

#frmat Initial List Status, Application Type
test['Initial List Status'] = np.where(test['Initial List Status']=='w', 1, 0)
test['Application Type'] = np.where(test['Application Type']=='INDIVIDUAL', 1, 0)


test_column_list = test.columns
for column in test_column_list:
    print('Number of null recs in col ',column,'is ',test[column].isnull().sum())


In [ ]:
#use earlier fit one model
test_data_ohc_encoded = pd.DataFrame (ohe.transform(test[categorical_cols]))

test_data_ohc_encoded.columns = ohe.get_feature_names(categorical_cols)

test.drop(categorical_cols ,axis=1, inplace=True)

test= pd.concat([test, test_data_ohc_encoded ], axis=1)

In [ ]:
y_col = 'Loan Status'

feature_cols_test = [x for x in test.columns if x != y_col]
X_test = test[feature_cols_test]
y_test_predict = grid.predict(X_test)

In [ ]:
final_submission = pd.DataFrame( list(zip(y_test_predict)),columns =['Loan Status'])

final_submission.to_csv('sub1.csv',index=False,header=True)